In [29]:
cd ~/notebooks/setup
source ./.env.sh

[sudo] password for aibiz: 


### MinIO + PostgreSQL 구성
- MinIO      : 2024-11-07T00-52-20Z
- PostgreSQL : 14.15
- History    :
  - MinIO서비스로 기동시 오류가 발생 
    - Type=oneshot
    - RemainAfterExit=yes
    - User/Group 제거
  - PostgreSQL 도커내 postgres가 생성한 폴더는 Host에서 다른 유저 권한으로 매핑
    - 도커 이미지의 postgres(999)를 aibiz의 UID(1000):GID(1000)으로 변환해서 기동

##### 1.1. 준비작업

In [359]:
# MinIO 설치폴더
sudo mkdir -p /opt/minio

# MinIO 저장소
sudo mkdir -p /data/minio/minio1-1 /data/minio/minio1-2
sudo mkdir -p /data/minio/minio2-1 /data/minio/minio2-2
sudo mkdir -p /data/minio/minio3-1 /data/minio/minio2-2
sudo mkdir -p /data/minio/minio4-1 /data/minio/minio2-2

# postgresql 폴더 생성
sudo mkdir -p /data/postgresql

# 권한 변경
sudo chown -R aibiz:aibiz /opt/minio /data/minio* /data/postgresql

# 네트워크 생성(없을 경우 1회만 실행)
cd /home/aibiz/notebooks/setup/minio
sudo docker network create --driver bridge data-network

In [ ]:
# mc client 설치
wget https://dl.min.io/client/mc/release/linux-amd64/mc
chmod +x mc
sudo mv mc /usr/local/bin/

##### 1-2. 기존 환경 백업

In [436]:
# MinIO 연결
mc alias set myminio http://localhost:7078 admin aibiz12#$

]11;?\Added `myminio` successfully.


In [43]:
# MinIO 백업
sudo mkdir -p /data/backup/minio_backup
mc mirror myminio /data/backup/minio_backup

...ummary.txt: 971.49 MiB / 971.49 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 1.31 GiB/s 0s


In [ ]:
# PostgreSQL 전체 백업 : 쉘에서 직접 실행할 것
sudo -i -u postgres
pg_dumpall -U postgres > /tmp/postgres_full_backup.sql
exit

In [5]:
# 백업 파일 이동 : 도커가 접근가능한 경로
sudo mv /tmp/postgres_full_backup.sql /data/backup/postgres_full_backup.sql
sudo chown -R aibiz:aibiz /data/backup

In [ ]:
# 원래 서비스 MinIO, PostgreSQL 종료
sudo systemctl stop minio
sudo systemctl stop postgresql

# 자동시작 제거
sudo systemctl disable minio
sudo systemctl disable postgresql

##### 1-3. 도커 서비스 기동 및 점검

In [30]:
# 서비스 종료
cd ~/notebooks/setup/minio
sudo docker-compose down

WARN[0000] Warning: No resource found to remove for project "minio". 


In [39]:
# 서비스 시작
cd ~/notebooks/setup/minio
sudo docker-compose up -d

[+] Running 1/0
 ⠋ Container minio-minio2-1    Starting                                    0.1s 
 ⠋ Container minio-minio3-1    Starting                                    0.1s 
 ⠋ Container minio-minio1-1    Starting                                    0.1s 
 ⠋ Container minio-postgres-1  Starting                                    0.1s 
 ⠋ Container minio-minio4-1    Starting                                    0.1s 
 ✔ Container minio-nginx-1     Created                                     0.0s 
[+] Running 1/6
 ⠙ Container minio-minio2-1    Starting                                    0.2s 
 ⠙ Container minio-minio3-1    Starting                                    0.2s 
 ⠙ Container minio-minio1-1    Starting                                    0.2s 
 ⠙ Container minio-postgres-1  Starting                                    0.2s 
 ⠙ Container minio-minio4-1    Starting                                    0.2s 
 ✔ Container minio-nginx-1     Created                                     0.

In [56]:
# 프로세스 확인
cd ~/notebooks/setup/minio
sudo docker-compose ps

NAME               IMAGE                                              COMMAND                  SERVICE    CREATED              STATUS                        PORTS
minio-minio1-1     quay.io/minio/minio:RELEASE.2024-12-13T22-19-12Z   "/usr/bin/docker-ent…"   minio1     About a minute ago   Up About a minute (healthy)   7077-7078/tcp, 9000/tcp
minio-minio2-1     quay.io/minio/minio:RELEASE.2024-12-13T22-19-12Z   "/usr/bin/docker-ent…"   minio2     About a minute ago   Up About a minute (healthy)   7077-7078/tcp, 9000/tcp
minio-minio3-1     quay.io/minio/minio:RELEASE.2024-12-13T22-19-12Z   "/usr/bin/docker-ent…"   minio3     About a minute ago   Up About a minute (healthy)   7077-7078/tcp, 9000/tcp
minio-minio4-1     quay.io/minio/minio:RELEASE.2024-12-13T22-19-12Z   "/usr/bin/docker-ent…"   minio4     About a minute ago   Up About a minute (healthy)   7077-7078/tcp, 9000/tcp
minio-nginx-1      nginx:1.19.2-alpine                                "/docker-entrypoint.…"   nginx      About a

In [77]:
# 웹서비스 기동 확인
sudo docker logs minio-nginx-1

/docker-entrypoint.sh: /docker-entrypoint.d/ is not empty, will attempt to perform configuration
/docker-entrypoint.sh: Looking for shell scripts in /docker-entrypoint.d/
/docker-entrypoint.sh: Launching /docker-entrypoint.d/10-listen-on-ipv6-by-default.sh
10-listen-on-ipv6-by-default.sh: Getting the checksum of /etc/nginx/conf.d/default.conf
10-listen-on-ipv6-by-default.sh: Enabled listen on IPv6 in /etc/nginx/conf.d/default.conf
/docker-entrypoint.sh: Launching /docker-entrypoint.d/20-envsubst-on-templates.sh
/docker-entrypoint.sh: Configuration complete; ready for start up


In [11]:
# 네트워크 확인
sudo docker network ls

NETWORK ID     NAME              DRIVER    SCOPE
aa406d47b8e5   airflow_default   bridge    local
89525f08bb7a   bridge            bridge    local
cdf418cb708d   data-network      bridge    local
403152440e5e   host              host      local
6d5aebbc4575   minio_default     bridge    local
a8a6c828591a   none              null      local


In [ ]:
# 네트워크 삭제(미사용)
echo "Y" | sudo docker network prune
sudo docker network create --driver bridge data-network

In [60]:
# 볼륨 확인
sudo docker volume ls

DRIVER    VOLUME NAME
local     1bc55fb42221cc6b86e6a3b3cfd5e64f533ad42a7d781d8bb786198b59d9ded9


In [13]:
# 볼륨 삭제(미사용)
echo "Y" | sudo docker volume prune

WARNING! This will remove anonymous local volumes not used by at least one container.
Are you sure you want to continue? [y/N] Total reclaimed space: 0B


In [14]:
# 기동여부 확인
curl http://localhost:7078

<?xml version="1.0" encoding="UTF-8"?>
<Error><Code>AccessDenied</Code><Message>Access Denied.</Message><Resource>/</Resource><RequestId>1813707CB433E527</RequestId><HostId>792e083e7334a5e09ca5da780329c569949338b18c30a3b5bde27d0221c62199</HostId></Error>


##### 1-4. 기존자료 복원

In [406]:
# MinIO 복구 (overwrite)
mc mirror /data/backup/minio_backup myminio --overwrite

...ummary.txt: 971.49 MiB / 971.49 MiB ┃▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓▓┃ 1.00 GiB/s 0s


In [407]:
# 참고 : 사용자 추가
mc admin user add myminio aibiz aibiz12#$
mc admin policy attach myminio readwrite --user aibiz

# 참고 : Delta Lake, Mlflow저장소 버킷 생성
mc mb myminio/dutchboy.db
mc mb myminio/mlflow.db

# 참고 : MinIO 복구 (not overwrite)
mc mirror /data/backup/minio_backup myminio

]11;?\Added user `aibiz` successfully.
]11;?\Attached Policies: [readwrite]
To User: aibiz


In [73]:
# PostgreSQL DB 복원
sudo docker exec -i minio-postgres-1 psql -U postgres -f /data/backup/postgres_full_backup.sql

SET
SET
SET
CREATE ROLE
ALTER ROLE
CREATE ROLE
ALTER ROLE
psql:/data/backup/postgres_full_backup.sql:18: ERROR:  role "postgres" already exists
ALTER ROLE
You are now connected to database "template1" as user "postgres".
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE DATABASE
ALTER DATABASE
You are now connected to database "airflow_db" as user "postgres".
SET
SET
SET
SET
SET
 set_config 
------------
 
(1 row)

SET
SET
SET
SET
CREATE SCHEMA
ALTER SCHEMA
SET
SET
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
ALTER SEQUENCE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
ALTER SEQUENCE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
ALTER SEQUENCE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
ALTER SEQUENCE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
ALTER SEQUENCE
CREATE TABLE
ALTER TABLE
CREATE SEQUENCE
ALTER TABLE
ALTER SEQUENCE
CREATE TABLE
ALTER 

##### 1-5. 서비스 등록

In [80]:
# 서비스 파일 생성
sudo cp ./minio.service /etc/systemd/system/.
sudo systemctl daemon-reload

In [91]:
# 서비스 시작
sudo systemctl start minio

In [9]:
# 서비스 종료
sudo systemctl stop minio

In [11]:
sudo systemctl stop postgresql

In [18]:
# 시스템 부팅 시 자동 시작 설정	
sudo systemctl enable minio

In [92]:
# 서비스 확인
sudo systemctl status minio --no-pager

● minio.service - MinIO Distributed Object Storage(with Docker)
     Loaded: loaded (]8;;file://aibiz-ai-rnd6/etc/systemd/system/minio.service/etc/systemd/system/minio.service]8;;; enabled; preset: enabled)
     Active: active (exited) since Sun 2024-12-22 09:10:39 KST; 5s ago
    Process: 2781372 ExecStart=docker-compose up -d (code=exited, status=0/SUCCESS)
   Main PID: 2781372 (code=exited, status=0/SUCCESS)
        CPU: 81ms

Dec 22 09:10:38 aibiz-ai-rnd6 docker-compose[2781372]:  Container minio-minio…ng
Dec 22 09:10:38 aibiz-ai-rnd6 docker-compose[2781372]:  Container minio-minio…ng
Dec 22 09:10:38 aibiz-ai-rnd6 docker-compose[2781372]:  Container minio-minio…ng
Dec 22 09:10:39 aibiz-ai-rnd6 docker-compose[2781372]:  Container minio-minio…ed
Dec 22 09:10:39 aibiz-ai-rnd6 docker-compose[2781372]:  Container minio-minio…ed
Dec 22 09:10:39 aibiz-ai-rnd6 docker-compose[2781372]:  Container minio-minio…ed
Dec 22 09:10:39 aibiz-ai-rnd6 docker-compose[2781372]:  Container minio-mini